In [ ]:
from manim import *
import CubicEquationSolver as ces
import moviepy.editor as mp
from IPython.display import Video

In [ ]:
def inizio(scene):
    title = Text("Dimostrazione del teorema della media")
    scene.play(Write(title))
    return title

In [ ]:
def teoremaValoriIntermedi(scene, old):
    title_text = Text("Premessa: \nTeorema dei valori \nintermedi").scale(1.4)
    enunciato = Text("Una funzione continua in [a,b]\nassume almeno una volta tutti i\nvalori compresi tra il suo minimo \ne il suo massimo in [a,b]")
    enunciato.scale(0.8)
    x_range = (-3, 9)
    y_range = (-3, 9)
    numberplane = NumberPlane(x_range=x_range, y_range=y_range, axis_config={"numbers_to_include": [i for i in range(*x_range)]})
    group = VGroup(title_text, enunciato)

    numberplane.set(width=config["frame_width"] /2, height=config["frame_height"])
    group.set(width=config["frame_width"] /3)
    group.arrange(DOWN, aligned_edge=LEFT, center=False)
    group.to_corner(UL)
    numberplane.to_corner(DR)
    scene.play(Transform(old, title_text))
    scene.wait(2)
    scene.play(Write(enunciato))

    under_func = lambda x: x**3 -2*x**2 -x +4
    derivata = lambda x: 3*x**2 -4*x -1
    # find zeros of the derivative
    a, b, c = 3, -4, -1
    delta = b**2 - 4*a*c
    x1 = (-b + delta**0.5)/(2*a)
    x2 = (-b - delta**0.5)/(2*a)
    min = under_func(x1)
    max = under_func(x2)

    interval = (-1, 2)
    # add vertical dashed lines at the endpoints of the interval, infinite height
    v_line1 = DashedLine(numberplane.coords_to_point(interval[0], y_range[0]), numberplane.coords_to_point(interval[0], y_range[1]), color=RED, dash_length=0.2)
    v_line2 = DashedLine(numberplane.coords_to_point(interval[1], y_range[0]), numberplane.coords_to_point(interval[1], y_range[1]), color=RED, dash_length=0.2)
    # add horizontal dashed lines at the min and max of the function, infinite width
    h_line1 = DashedLine(numberplane.coords_to_point(x_range[0], min), numberplane.coords_to_point(x_range[1], min), color=GREEN, dash_length=0.2)
    h_line2 = DashedLine(numberplane.coords_to_point(x_range[0], max), numberplane.coords_to_point(x_range[1], max), color=GREEN, dash_length=0.2)
    # add a dot at the min and max of the function
    dot1 = Dot(numberplane.coords_to_point(x1, min), color=RED)
    dot2 = Dot(numberplane.coords_to_point(x2, max), color=RED)
    
    #func.shift(numberplane.coords_to_point(0, 0))
    func = numberplane.plot(under_func, color=GREEN)
    scene.add(numberplane, v_line1, v_line2, h_line1, h_line2, dot1, dot2, func)
    scene.wait(2)
    
    # zoom in on the interval
    scene.camera.frame.save_state()
    scene.play(scene.camera.frame.animate(run_time= 3).set(width=7).move_to(numberplane.coords_to_point(0.5, (max+min)/2)))

    move_line = Line(numberplane.coords_to_point(interval[0], min), numberplane.coords_to_point(interval[1], min), color=RED)
    scene.play(Create(move_line))
    # move some dots on the function as the line moves
    dots = [Dot(numberplane.coords_to_point(0, min), color=RED, radius=0.1) for _ in range(3)]
    def line_updater(dot):
        index = dots.index(dot)
        y = numberplane.point_to_coords(move_line.get_start())[1]
        x = ces.solve(1, -2, -1, 4-y)
        # remove x outside the interval
        x = [i for i in x if i >= interval[0] and i <= interval[1]]
        # move the dot
        if index >= len(x):
            index = len(x) - 1
        dot.move_to(numberplane.coords_to_point(x[index], y))
    for dot in dots:
        dot.add_updater(line_updater)
    scene.add(*dots)
    # move the line to the max
    scene.play(move_line.animate(run_time= 3).shift(numberplane.coords_to_point(interval[0], max) - move_line.get_start()))
    scene.wait(1)
    scene.clear()
    scene.play(Restore(scene.camera.frame))

    

In [ ]:
def dimostrazione(scene):
    weier = Text("Per il teorema di Weierstrass:")
    start = MathTex("m","<=","f(x)", "<=", "M")
    propIntDef = Text("Per la proprietà dell'integrale definito:").scale(0.8)
    integrato = MathTex("\\int_{a}^{b} m \\,dx\\", "<=", "\\int_{a}^{b} f(x) \\,dx\\", "<=", "\\int_{a}^{b} M \\,dx\\")
    sempre = Text("Sempre per la proprietà dell'integrale definito:").scale(0.8)
    raccolto = MathTex("m(b-a)", "<=", "\\int_{a}^{b} f(x) \\,dx\\", "<=", "M(b-a)")
    divido = Tex("Divido per $(b-a)$:").scale(0.9)
    diviso = MathTex("m", "<=", "\\frac{\\int_{a}^{b} f(x) \\,dx}{b-a}", "<=", "M")

    group = VGroup(weier, start, integrato, raccolto, diviso)
    group.arrange(DOWN, buff=LARGE_BUFF)
    group.scale(0.7)
    group.to_edge(UP)
    divido.to_edge(UP)
    sempre.to_edge(UP)
    propIntDef.to_edge(UP)

    play_kw = {"run_time": 2}

    # per il teorema di weierstrass
    scene.play(Write(weier))
    scene.play(Write(start))
    scene.wait(7)
    # propIntDef to same position of weier

    # integro tutta la disequazione
    scene.play(Transform(weier, propIntDef))
    scene.play(
        TransformMatchingTex(
            start.copy(), integrato,
            transform_mismatches=True,
            key_map={
                "f(x)": "\\int_{a}^{b} f(x) \\,dx\\",
                "m": "\\int_{a}^{b} m \\,dx\\"
            }
        ),
        **play_kw
    )
    scene.wait(10)

    # cambio gli integrali del min max per il significato geometrico
    scene.play(Transform(weier, sempre, use_override=False))
    scene.play(
        TransformMatchingTex(
            integrato.copy(), raccolto,
            transform_mismatches=True,
            key_map={
                "\\int_{a}^{b} m \\,dx\\": "m(b-a)",
                "\\int_{a}^{b} M \\,dx\\": "M(b-a)"
            },
        ),
        **play_kw
    )
    scene.wait(10)

    # divido per b-a
    scene.play(Transform(weier, divido))
    scene.play(
        TransformMatchingTex(
            raccolto.copy(), diviso,
            transform_mismatches=True,
            key_map={
                "\\int_{a}^{b} f(x) \\,dx\\": "\\frac{\\int_{a}^{b} f(x) \\,dx}{b-a}",
                "m(b-a)": "m",
                "M(b-a)": "M"
            },
            use_override=False
        ),
        **play_kw
    )
    scene.wait(6)

    # tolgo tutto tranne diviso
    scene.play(FadeOut(weier), FadeOut(start), FadeOut(integrato), FadeOut(raccolto))
    # center and scale diviso
    scene.play(diviso.animate.scale(2).move_to(ORIGIN))
    valori_intermedi = Text("Per il teorema dei valori intermedi")
    valori_intermedi.to_edge(UP)

    scene.wait(12)
    scene.play(Write(valori_intermedi))

In [ ]:
%%manim -qh -v WARNING DimostrazioneTeoremaMedia 
class DimostrazioneTeoremaMedia(MovingCameraScene):
    def construct(self):
        title = inizio(self)
        teoremaValoriIntermedi(self, title)
        dimostrazione(self)
        self.wait(8)


In [ ]:
audio = mp.AudioFileClip(r"C:\Users\samue\OneDrive\Documenti\Registrazioni di suoni\Registrazione in corso (2).mp3")
clip = mp.VideoFileClip("media/videos/Python Scripts/1080p60/DimostrazioneTeoremaMedia.mp4")
out = clip.set_audio(audio)
out.write_videofile("out.mp4")
Video("out.mp4")